In [1]:
import sys
sys.path += ['/Users/cswa648/git/phylo-hacking/pymc']

import newick
import theano
%matplotlib inline

In [2]:
newick_string = '((((ANDI:15,DIGI:15):50,((CASS:35,EMU:35):10,KIWI:45):10):10,RHEA:55):20,OST:85);'
tree = newick.loads(newick_string)[0]
print(tree.ascii_art())

                            ┌─ANDI
                     ┌──────┤
                     │      └─DIGI
              ┌──────┤
              │      │             ┌─CASS
              │      │      ┌──────┤
       ┌──────┤      └──────┤      └─EMU
       │      │             └─KIWI
───────┤      │
       │      └─RHEA
       └─OST


In [32]:
import pymc3 as pm

from importlib import reload
import pylo.tree
reload(pylo.tree)
import pylo.topology
reload(pylo.topology)

from pylo.topology import TreeTopology
from pylo.tree import BirthDeathSamplingTree
from pylo.hky import HKYSubstitutionModel

topology = pylo.topology.TreeTopology(tree)

with pm.Model() as model:
    r = pm.Exponential('r', lam=0.1)
    a = pm.Beta('a', alpha=1.0, beta=1.0)
    rho = pm.Beta('rho', alpha=1.0, beta=1.0)
    
    tree_heights = BirthDeathSamplingTree('tree', topology, r, a, rho)
    branch_lengths = pm.Deterministic('branch_lengths', topology.get_child_branch_lengths(tree_heights))
    rates = pm.Lognormal('rates', size=branch_lengths.shape)
    
    distances = pm.Deterministic('distances', rates*branch_lengths)
    
    kappa = pm.Exponential('kappa', lam=0.1)
    pi = pm.Dirichlet('pi', a=np.ones(4))
    
    substitution_model = HKYSubstitutionModel(kappa, pi)
    
model